In [86]:
import os
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

In [87]:
# Load the IMDB Reviews dataset
imdb = pd.read_csv("../data/movie/IMDB.csv")

In [88]:
imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [89]:
imdb['sentiment'] = imdb['sentiment'].map({'positive': 1, 'negative': 0})

In [90]:
imdb.shape

(50000, 2)

In [91]:
X = imdb["review"]
y = imdb["sentiment"]

In [92]:
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(X,y,test_size=0.5,random_state=42)

In [93]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Generate Padded Sequences

In [94]:
# Parameters
vocab_size = 10000
max_length = 120
embedding_dim = 16
trunc_type='post'
oov_tok = "<OOV>"

In [95]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length, truncating=trunc_type)

### Build and Compile the Model

In [96]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [97]:
#summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_2 (Flatten)         (None, 1920)              0         
                                                                 
 dense_4 (Dense)             (None, 6)                 11526     
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


### Train the Model

In [98]:
num_epochs = 10

# Train the model
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5031 - accuracy: 0.7352 - val_loss: 0.3635 - val_accuracy: 0.8386
Epoch 2/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2429 - accuracy: 0.9056 - val_loss: 0.3807 - val_accuracy: 0.8339
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0872 - accuracy: 0.9787 - val_loss: 0.4679 - val_accuracy: 0.8229
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0198 - accuracy: 0.9977 - val_loss: 0.5464 - val_accuracy: 0.8260
Epoch 5/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.6105 - val_accuracy: 0.8276
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0019 - accuracy: 0.9999 - val_loss: 0.6619 - val_accuracy: 0.8269
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 8.6359e-04 - accuracy: 1.0000 - val_loss: 0.7078 - val_accuracy: 0.8272
Ep